# Exploring *Trans* Effects of Copy Number Amplifications Across Cancer Types

**Author:** Avery Bell, Brigham Young University

**Contact:** avery.mecham@outlook.com

---
This notebook explores the results of linear regressions between the CNV data for 307 genes and all the proteins listed in CPTAC (see [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5117628/) for more information on the dataset. This notebook falls as the third step in our process, after [finding the significant CNVs](https://github.com/dlewis27/CNVpan-cancer/tree/master/findingCNVs) and [performing the linear regressions](https://github.com/dlewis27/CNVpan-cancer/blob/master/transEffects/workflow.txt). The main purpose of this notebook is to (1) find gene/protein pairs from the regressions that are significant across cancers and (2) to determine if any significant gene/protein pairs show a positive slope in one cancer and a negative slope in another.

Note: Please download [this directory](https://github.com/dlewis27/CNVpan-cancer/tree/master/transEffects) and run this code with the working directory set to the same directory as your download because there are some files necessary for the code to run.

## Load Libraries
Loads the necessary libraries for this notebook to run.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import glob, os
import numpy as np
import seaborn as sns
from matplotlib.gridspec import GridSpec

## Load Gene/Protein Regressions
Loads the files in the directory BonCorrect, which should each be tab-delimited and contain the word "up" (indicating that the genes they reference were amplified). These files are derived from the script located [here](https://github.com/dlewis27/CNVpan-cancer/blob/master/transEffects/runLinearRegressions.py).

In [0]:
up_regress = pd.DataFrame({})

for f in os.listdir('BonCorrect'):
  if f.find('up') > 0:
    this_tsv = pd.read_csv('BonCorrect/' + f, delimiter='\t', encoding='utf-8')
    cancer_type = f.split('_')[0]
    this_tsv['cancer_type'] = [cancer_type for _ in range(this_tsv.shape[0])]
    up_regress = pd.concat([up_regress, this_tsv])

up_regress.head()

## Count Unique Gene/Protein Pairs

In [0]:
np.unique(up_regress.iloc[:,[1, 2]]).shape[0]

## Load CNV Data
Loads the files in the directory CNV Tables that contain copy number amplifications significant in each cancer, as described [here](https://github.com/dlewis27/CNVpan-cancer/blob/master/findingCNVs/notes.txt).

In [0]:
up_cnv = {}

for f in os.listdir('CNV Tables'):
  if f.find('.txt') > 0 and f.find('up') > 0:
    this_tsv = pd.read_csv('CNV Tables/' + f, delimiter='\t', encoding='utf-8', index_col=0)
    cancer_type = f.split('_')[0]
    up_cnv[cancer_type] = this_tsv

print(up_cnv)

## Load Protein Data
Loads the protein data used in the linear regressions. Protein data should be in a directory called "Protein tables" and each file should include the cancer type (in the form of its symbol in CPTAC, e.g., head and neck squamous cell carcinoma is hnscc).

In [0]:
up_protein = {}

for f in os.listdir('Protein tables 1'):
  this_tsv = pd.read_csv('Protein tables 1/' + f, delimiter='\t', encoding='utf-8')
  cancer_type = f.split('ProData')[0]
  if 'Patient_ID' in this_tsv.columns:
    this_tsv = this_tsv.set_index('Patient_ID')
  else:
    this_tsv = this_tsv.set_index('Name')
  up_protein[cancer_type] = this_tsv

for f in os.listdir('Protein tables 2'):
  this_tsv = pd.read_csv('Protein tables 2/' + f, delimiter='\t', encoding='utf-8')
  cancer_type = f.split('ProData')[0]
  if 'Patient_ID' in this_tsv.columns:
    this_tsv = this_tsv.set_index('Patient_ID')
  else:
    this_tsv = this_tsv.set_index('Name')
  up_protein[cancer_type] = this_tsv

print(up_protein)

## Helper Functions
This function allows the user to easily plot the regression line associated with a given gene/protein pair in a given cancer type.

In [0]:
def plot_points(data):
  cnv_name = data.CNV
  cancer_type = data.cancer_type
  CNV_data = up_cnv[cancer_type][cnv_name]
  prot_name = data.Protien
  protein_data = up_protein[cancer_type][prot_name]

  low_merged = pd.merge(CNV_data, protein_data, how = 'inner', right_index=True, left_index=True)

  low_merged.plot.scatter(x = cnv_name, y = prot_name)

  x = np.linspace(min(CNV_data),max(CNV_data),100)
  y = data.slope * x + data.intercept
  plt.plot(x, y, '-r', label='line of best fit')
  plt.title("Cancer: " + cancer_type + ", Gene: " + cnv_name + ", Protein: " + prot_name + ", R Value: " + str(data.rValue))

## Gene/Protein Pairs Common Across Cancers
Creates a table with rows as cancer types and columns as gene/protein pairs. Cancer types with a gene/protein pair that was not significant for that cancer (as determined by the Bonferroni-corrected p-value, see [this file](https://github.com/dlewis27/CNVpan-cancer/blob/master/transEffects/workflow.txt)) will appear as an `NA` value in the row for that cancer.

In [0]:
regress_pivot = up_regress[['CNV', 'Protien', 'slope', 'cancer_type']]
regress_pivot['gene_prot_pair'] = regress_pivot[['CNV', 'Protien']].apply(lambda x: '/'.join(x.dropna().astype(str).values), axis=1)
regress_pivot = regress_pivot.pivot(index='cancer_type',columns='gene_prot_pair', values='slope')

To get the gene/protein pairs that were significant in at least six of the seven cancers, we drop those with more than one `NA` value (i.e., no more than one cancer can have missing data for that gene/protein pair). Then we drop the one cancer that had `NA` values for those gene/protein pairs for which the rest of the cancers had data.

In [0]:
sig_regress_pivot = regress_pivot.loc[:, regress_pivot.isna().sum() < 2]
sig_regress_pivot = sig_regress_pivot.dropna()

We see that this filtering results in 62 gene/protein pairs that are significantly correlated across six of the seven cancers.

In [0]:
print(len(np.unique(sig_regress_pivot.columns)))

We see that the one cancer that does not show significant correlation for these 62 gene/protein pairs is GBM, which we note is missing from the list.

In [0]:
print(np.unique(sig_regress_pivot.index))

In a heatmap, we visualize the slope of the regressions for the 62 gene/protein pairs and notice that there are only two proteins represented: NUDCD1 and YHWAZ.

In [0]:
cg = sns.clustermap(sig_regress_pivot)
cg.ax_col_dendrogram.set_visible(False)

cg.ax_heatmap.set_ylabel('Cancer Type')
cg.ax_heatmap.set_xlabel('CNV/Protein Pair')

plt.figure(figsize=(30, 7))

We visualize NUDCD1 specifically.

In [0]:
cm = sns.clustermap(sig_regress_pivot.loc[:,[p.find("NUDCD1") >= 0 for p in sig_regress_pivot.columns]], figsize=(15, 8), cbar_pos=(0, 0, .05, .3))

cm.ax_col_dendrogram.set_visible(False)

cm.ax_heatmap.set_ylabel('Cancer Type')
cm.ax_heatmap.set_xlabel('CNV/Protein Pair')
plt.title("slope") 

We visualize YWHAZ specifically. This one doesn't show a legend because it will be combined with the NUDCD1 figure for the publication.

In [0]:
cm2 = sns.clustermap(sig_regress_pivot.loc[:,[p.find("YWHAZ") >= 0 for p in sig_regress_pivot.columns]], figsize=(15, 8), cbar_pos=None)

cm2.ax_col_dendrogram.set_visible(False)

cm2.ax_heatmap.set_ylabel('Cancer Type')
cm2.ax_heatmap.set_xlabel('CNV/Protein Pair')

## Differential *Trans* Effects Between Cancers
Here we determine whether there is a gene/protein pair that is significantly positively correlated in one cancer and significantly negatively correlated in another. We can see that of the significant gene/protein pairs, there are none.

In [0]:
# Get gene-protein pairs
cnv_prot_pairs = up_regress.loc[:, ['CNV', 'Protien']].drop_duplicates()
# For every pair
for index, row in cnv_prot_pairs.iterrows():
  # Get slope > 0 cancers
  up_slope = set(up_regress.loc[(up_regress.CNV == row['CNV']) & (up_regress.Protien == row['Protien']) & (up_regress.slope > 0)].cancer_type)
  # Get slope < 0 cancers
  down_slope = set(up_regress.loc[(up_regress.CNV == row['CNV']) & (up_regress.Protien == row['Protien']) & (up_regress.slope < 0)].cancer_type)
  # Is there a cancer that is different between the lists
  if len(up_slope) > len(down_slope):
    diff = down_slope - up_slope
  else:
    diff = up_slope - down_slope
  
  if len(diff) > 0:
    print('CNV: {}, Protein: {}'.format(row['CNV'], row['Protien']))
    print(diff)

## Why Are Some Regression Slopes Negative?
The finding above leads us to ask which cancer is the source of some negative slopes that we see in the significant gene/protein pairs.

Here we see the distribution of all slopes for all regressions and all cancers. Though most gene/protein pairs have a positive slope, a cluster falls below zero.

In [0]:
for c_type in up_regress.cancer_type.unique():
  sns.distplot(up_regress.loc[up_regress.cancer_type == c_type].slope)
plt.legend(labels=up_regress.cancer_type.unique(), title="Cancer Type")
plt.xlabel("Slope")
plt.ylabel("Number of Gene/Protein Pairs")

We investigate the negative slopes piece by piece, subsetting by the most significant pathways in each GSEA result for better clarity on what is going on. The first GSEA result is from GO Molecular Function.

In [0]:
go_molecular_function = pd.read_csv('../pathways/GO_Molecular_Function.csv')
first_genes3 = go_molecular_function.iloc[0].Genes.split(';')
for g in first_genes3:
  sns.distplot(up_regress.loc[up_regress.CNV == g].slope)
plt.legend(labels=first_genes3)

We can see that BRCA is responsible for the negative slopes.

In [0]:
for c in first_genes3:
  print(c)
  print(np.unique(up_regress.loc[(up_regress.CNV == c) & (up_regress.slope > 0)].cancer_type))
  print(np.unique(up_regress.loc[(up_regress.CNV == c) & (up_regress.slope < 0)].cancer_type))
  print(np.unique(up_regress.loc[(up_regress.CNV == c) & (up_regress.slope == 0)].cancer_type))

Then we visualize the Reactome results.

In [0]:
reactome_2016 = pd.read_csv('../pathways/Reactome_2016.csv')
first_genes4 = reactome_2016.iloc[0].Genes.split(';')
for g in first_genes4:
  sns.distplot(up_regress.loc[up_regress.CNV == g].slope)
plt.legend(labels=first_genes4)

We see that BRCA, endometrial, and LUAD are responsible for the negative slopes. SEC61 only appears to be differentially correlated because SEC61 does not appear in any cancer except BRCA.

In [0]:
for c in first_genes4:
  print(c)
  print(np.unique(up_regress.loc[(up_regress.CNV == c) & (up_regress.slope > 0)].cancer_type))
  print(np.unique(up_regress.loc[(up_regress.CNV == c) & (up_regress.slope < 0)].cancer_type))
  print(np.unique(up_regress.loc[(up_regress.CNV == c) & (up_regress.slope == 0)].cancer_type))